# AnalyticDB

>[AnalyticDB for PostgreSQL](https://www.alibabacloud.com/help/en/analyticdb-for-postgresql/latest/product-introduction-overview) 是一个大规模并行处理 (MPP) 数据仓库服务，专为在线分析海量数据而设计。

>`AnalyticDB for PostgreSQL` 基于开源的 `Greenplum Database` 项目开发，并由 `Alibaba Cloud` 进行深度增强。AnalyticDB for PostgreSQL 兼容 ANSI SQL 2003 语法以及 PostgreSQL 和 Oracle 数据库生态系统。AnalyticDB for PostgreSQL 还支持行存储和列存储。AnalyticDB for PostgreSQL 以高性能处理 PB 级别的数据离线分析，并支持高并发的在线查询。

您需要安装 `langchain-community` 才能使用此集成，命令为 `pip install -qU langchain-community`

本 Notebook 演示了如何使用与 `AnalyticDB` 向量数据库相关的功​​能。
要运行，您应该有一个正在运行的 [AnalyticDB](https://www.alibabacloud.com/help/en/analyticdb-for-postgresql/latest/product-introduction-overview) 实例：

- 使用 [AnalyticDB Cloud Vector Database](https://www.alibabacloud.com/product/hybriddb-postgresql)。点击此处快速部署。

In [ ]:
from langchain_community.vectorstores import AnalyticDB
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

通过调用 OpenAI API 分割文档并获取嵌入

In [2]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("../../how_to/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

通过设置相关环境变量来连接 AnalyticDB。
```
export PG_HOST={your_analyticdb_hostname}
export PG_PORT={your_analyticdb_port} # 可选，默认为 5432
export PG_DATABASE={your_database} # 可选，默认为 postgres
export PG_USER={database_username}
export PG_PASSWORD={database_password}
```

然后将您的 embeddings 和 documents 存储到 AnalyticDB 中

In [3]:
import os

connection_string = AnalyticDB.connection_string_from_db_params(
    driver=os.environ.get("PG_DRIVER", "psycopg2cffi"),
    host=os.environ.get("PG_HOST", "localhost"),
    port=int(os.environ.get("PG_PORT", "5432")),
    database=os.environ.get("PG_DATABASE", "postgres"),
    user=os.environ.get("PG_USER", "postgres"),
    password=os.environ.get("PG_PASSWORD", "postgres"),
)

vector_db = AnalyticDB.from_documents(
    docs,
    embeddings,
    connection_string=connection_string,
)

查询和检索数据

In [4]:
query = "What did the president say about Ketanji Brown Jackson"
docs = vector_db.similarity_search(query)

In [5]:
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
